In [1]:
import pandas as pd

In [2]:
dataset_name = 'anda_fert_sintetico'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Prata

NPK na forma de fertilizante sintético incorporado ao solo (t)

In [3]:
tabelas = ['A_fert_N', 'A_fert_P', 'A_fert_K']

anos = (pd.read_excel('../../airflow-dags-isagro/isagro/resources/Balanco_NPK_1ed_04_02_23.xlsx',
                sheet_name='A_fert_N',
                header=4,
                nrows=0,
                index_col=0
                )
        .columns
        .tolist()
        )

In [4]:
for fertilizantes in tabelas:
      (pd.read_excel('../../airflow-dags-isagro/isagro/resources/Balanco_NPK_1ed_04_02_23.xlsx',
                sheet_name=fertilizantes,
                na_filter=False,
                header=4)
      .query('UF != "Brasil"')
      .merge(pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet')
              .filter(['UF-id', 'UF-nome'])
              .drop_duplicates(),
              how='left',
              left_on='UF',
              right_on='UF-nome'
              )
      .melt(id_vars='UF-id',
              value_vars=anos,
              var_name='ano',
              value_name='valor'
              )
      .dropna(subset='valor')
      .rename(columns={'UF-id':'uf'})
      .assign(**{
          'fonte': 'Fertilizantes Sintéticos',
          'ano': lambda x: x['ano'].astype(int)
          })
      .convert_dtypes()
      .sort_values(by=['uf', 'ano'])
      .set_index(['uf', 'ano', 'fonte'])
      .to_parquet(f'{raiz}/prata_{fertilizantes}.parquet')
      )

### Ouro

Elementos químicos na forma de fertilizante entregue ao mercado (t)

Parâmetros

Teor (%)

Cálculos:
Valor Estadual N * (Parâmetro N/100) * ((100 - fertilizantes_sinteticos)/100)

Valor Estadual P2o5 * (Parâmetro P)/100

Valor Estadual K2o * (Parâmetro K)/100


Brasil = mil t

In [5]:
parametro_N = 100
parametro_P = 43.7
parametro_K = 83

# Perdas por volatilização do N
perdas_fertiliz_sintetico = 0

In [6]:
df = (pd.read_parquet(f'{raiz}/prata_A_fert_N.parquet')
      .assign(**{
          'N': lambda df: df['valor'].apply(lambda t: t*(parametro_N/100)*((100 - perdas_fertiliz_sintetico)/100))
          })
      .drop('valor', axis = 1)
      .merge(pd.read_parquet(f'{raiz}/prata_A_fert_P.parquet')
             .assign(**{
                 'P': lambda df: df['valor'].apply(lambda t: t*(parametro_P/100))
                 })
             .drop('valor', axis = 1),
             how='inner',
             on=['uf', 'ano', 'fonte']
             )
      .merge(pd.read_parquet(f'{raiz}/prata_A_fert_K.parquet')
             .assign(**{
                 'K': lambda df: df['valor'].apply(lambda t: t*(parametro_K/100))
                 })
             .drop('valor', axis = 1),
             how='inner',
             on=['uf', 'ano', 'fonte']
             )
      .convert_dtypes()
      .reset_index()
      .melt(id_vars=['uf', 'ano', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
      .set_index(['uf', 'ano', 'fonte'])
      )

df.to_csv(f'{raiz}/ouro_npk_fert_sintetico.csv')